# Language Conditioned Robot Skills (LCRS) - Live Demo

## Load the pre-trained model

In [ ]:
from calvin_agent.utils.utils import print_system_env_info, get_last_checkpoint, format_sftp_path
from pytorch_lightning.utilities import rank_zero_only
from pytorch_lightning.loggers import Logger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import Callback, LightningModule, seed_everything, Trainer
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.utilities.cloud_io import load as pl_load
import hydra
import logging
from pathlib import Path
import sys
from typing import List
import lcrs


logger = logging.getLogger(__name__)


@hydra.main(config_path="../conf", config_name="config")
def predict(cfg: DictConfig) -> None:
    seed_everything(cfg.seed, workers=True)
    datamodule = hydra.utils.instantiate(cfg.datamodule, training_repo_root=Path(lcrs.__file__).parents[1])
    model = hydra.utils.instantiate(cfg.model)
    chk = get_last_checkpoint(Path.cwd())

    if chk:
        pretrain_chk = pl_load(format_sftp_path(Path(cfg.pretrain_chk)), map_location=lambda storage, loc: storage)
        model.load_state_dict(pretrain_chk["state_dict"], strict=False)
        
        log_rank_0("LOADED PRE-TRAINED MODEL")
    else:
        raise(Exception("No pre-trained model found"))
    
    log_rank_0(print_system_env_info())


    trainer_args = {
        **cfg.trainer,
        "logger": train_logger,
        "callbacks": callbacks,
        "benchmark": False,
    }

    trainer = Trainer(**trainer_args)

    predictions = trainer.predict(model, datamodule=datamodule)

@rank_zero_only
def log_rank_0(*args, **kwargs):
    logger.info(*args, **kwargs)


if __name__ == "__main__":
    # train()
    print("main")


# loaded_model = lcrs.load_from_checkpoint("~/runs/2024-03-07/21-32-22/saved_models/epoch=0.ckpt")



## Setup simulation environment

CALVIN

In [ ]:
%cd calvin
from hydra import initialize, compose

with initialize(config_path="./calvin_env/conf/"):
  cfg = compose(config_name="config_data_collection.yaml", overrides=["cameras=static_and_gripper"])
  cfg.env["use_egl"] = False
  cfg.env["show_gui"] = False
  cfg.env["use_vr"] = False
  cfg.env["use_scene_info"] = True
  print(cfg.env)

In [ ]:
import time
import hydra
import numpy as np
from google.colab.patches import cv2_imshow

env = hydra.utils.instantiate(cfg.env)
observation = env.reset()
#The observation is given as a dictionary with different values
print(observation.keys())
for i in range(5):
  # The action consists in a pose displacement (position and orientation)
  action_displacement = np.random.uniform(low=-1, high=1, size=6)
  # And a binary gripper action, -1 for closing and 1 for oppening
  action_gripper = np.random.choice([-1, 1], size=1)
  action = np.concatenate((action_displacement, action_gripper), axis=-1)
  observation, reward, done, info = env.step(action)
  rgb = env.render(mode="rgb_array")[:,:,::-1]
  cv2_imshow(rgb)

## Inference

In [ ]:
loaded_model.step() #todo: integrate environment into it and so on
predict
predict_step